# Breast Cancer Dataset

Binary Classification Prediction for type of Breast Cancer

https://www.kaggle.com/datasets/yasserh/breast-cancer-dataset?resource=download

In [1]:
!pip install scikit-learn
!pip install pandas

In [ ]:
import pandas as pd
from glob import glob

for g in glob('./*'):
    print(g)

df = pd.read_csv('./chap01/data/breast-cancer.csv')
print(df.shape)
df.head()

## 데이터 전처리

In [ ]:
# 비어있는게 있나 
print(df.isnull().sum())


In [ ]:
# 이상치 있는지 확인
df.describe()

In [ ]:
!pip install matplotlib

In [ ]:
# 전체 데이터 분포 그래프로 확인
import matplotlib.pyplot as plt

df.hist(bins=20, figsize=(20, 15))
plt.show()

# 이상치는 없는 것 같다. 

In [ ]:
# 양성, 음성으로 나눠 데이터 분포 확인

df_m = df[df['diagnosis'] == 'M']
df_b = df[df['diagnosis'] == 'B']

plt.figure(figsize=(20, 25))
for i, feature in enumerate(df.columns[2:32]):
    plt.subplot(6, 5, i+1)
    plt.hist(df_m[feature], bins=20, color='r', alpha=0.5, label='M')
    plt.hist(df_b[feature], bins=20, color='b', alpha=0.5, label='B')
    plt.legend()
    plt.title(feature)


In [ ]:
# M과 B의 숫자 확인

print(df['diagnosis'].value_counts())

plt.figure(figsize=(10, 5))
plt.bar(df['diagnosis'].value_counts().index, df['diagnosis'].value_counts().values)

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns

plt.figure(figsize=(20, 20))

sns.heatmap(df.drop(columns=['id', 'diagnosis']).corr(), annot=True, fmt='.2f', square=True, cmap='coolwarm')
plt.show()





다중 공선성을 처리해야 하는 변수들이 좀 보인다. 

일단은 무시하고 가보자. 

In [ ]:
df['diagnosis_bin'] = df['diagnosis'].map({'M': 1, 'B': 0})

df.drop(columns=['id', 'diagnosis'], inplace=True)

In [ ]:
df_origin = df.copy()



df_features = df.drop(columns=['diagnosis_bin'])
df_target = df['diagnosis_bin']


display(df_features.head())

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=5)

kn.fit(df_features, df_target)

In [ ]:
kn.score(df_features, df_target)

In [ ]:
df['predicted_with_all_features'] = kn.predict(df_features)
df.head()

In [ ]:
# 상관관계가 제일 높은 feature 2개는 뭐였을까? 
df.corr()['diagnosis_bin'].sort_values(ascending=False)

In [ ]:
# concave points_worst와 perimeter_worst을 x, y로 scatter plot 그리자. 
# 색상은 diagnosis_bin으로 구분
# predicted_with_all_features와 diagnosis_bin의 값이 다른 곳은 표시를 하자.

plt.scatter(df['concave points_worst'], df['perimeter_worst'], c=df['diagnosis_bin'])
plt.scatter(
    df[df['diagnosis_bin'] != df['predicted_with_all_features']]['concave points_worst'],
    df[df['diagnosis_bin'] != df['predicted_with_all_features']]['perimeter_worst'],
    marker='x',
)
plt.xlabel('predicted_with_all_features')
plt.ylabel('concave points_worst')
plt.show()

## train, test set을 나눠야 한다. 

In [ ]:
from sklearn.model_selection import train_test_split

df_features = df.drop(columns=['diagnosis_bin', 'predicted_with_all_features'])

X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.2, random_state=42, stratify=df_target)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(y_train.value_counts(), y_train.value_counts() / y_train.shape[0] * 100)
print(y_test.value_counts(), y_test.value_counts() / y_test.shape[0] * 100)

In [ ]:
# normalize
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# X_train_scaled 에 컬럼명 부여하기
df_X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
df_X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
# pairplot 해보자. 
df_train = pd.DataFrame(X_train_scaled, columns=df_features.columns)
df_train['diagnosis_bin'] = y_train

sns.pairplot(df_train, hue='diagnosis_bin')
plt.show()

In [ ]:
!pip install statsmodels

In [ ]:
# correlation heatmap 

plt.figure(figsize=(20, 20))
sns.heatmap(
    pd.DataFrame(X_train_scaled, columns=df_features.columns).corr(), annot=True, fmt='.2f', square=True, cmap='coolwarm'
)
# sns.heatmap(X_train_scaled.corr, annot=True, fmt='.2f', square=True, cmap='coolwarm')
plt.show()

In [ ]:
# 다중공선성 처리
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X_train_scaled, i) for i in range(X_train_scaled.shape[1])]
vif['features'] = df_X_train_scaled.columns
vif.sort_values('VIF Factor', ascending=False, inplace=True)
vif


VIF(Variance Inflation Factor)는 다중공선성을 평가하기 위한 통계적 지표입니다. 다중공선성은 독립 변수들 간에 강한 선형 관계가 있는 경우 발생하며, 이는 회귀 분석 결과의 신뢰성을 저하시킬 수 있습니다. VIF는 각 독립 변수의 설명력과 다른 독립 변수들과의 상관 관계를 측정하여 다중공선성을 평가합니다.

VIF 값은 1보다 크거나 같은 양수입니다. 일반적으로 VIF 값이 1보다 작으면 다중공선성이 거의 없다고 판단하고, 1보다 큰 값은 다중공선성이 존재한다고 판단합니다. 일반적으로 VIF 값이 5를 넘어가면 다중공선성이 심각하다고 판단합니다.

VIF를 활용하여 다중공선성을 제거하기 위한 방법은 다음과 같습니다:

1. 다중공선성이 의심되는 독립 변수들을 선택합니다.
2. 선택한 독립 변수들을 이용하여 다중회귀 분석을 수행합니다.
3. 각 독립 변수의 VIF 값을 계산합니다.
4. VIF 값이 일정 기준(예: 5)을 넘는 독립 변수를 제거합니다.
5. 제거한 독립 변수를 제외한 나머지 독립 변수들을 이용하여 다시 다중회귀 분석을 수행합니다.
6. 위 과정을 반복하여 VIF 값이 기준을 넘지 않는 최종 독립 변수들을 선택합니다.

이 방법을 통해 다중공선성이 있는 독립 변수들을 제거하고, 모델의 신뢰성을 향상시킬 수 있습니다.

In [ ]:
df_X_train_scaled.drop(columns=['perimeter_mean'], inplace=True)

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(df_X_train_scaled.to_numpy(), i) for i in range(df_X_train_scaled.shape[1])]
vif['features'] = df_X_train_scaled.columns
vif.sort_values('VIF Factor', ascending=False, inplace=True)
display(vif)